11.1 Create a file called zoo.py. In it, define a function called hours() that prints the string 'Open 9-5 daily'. Then, use the interactive interpreter to import the zoo module and call its hours() function.

In [5]:
import zoo
zoo.hours()

Open 9-5 daily.


11.2 In the interactive interpreter, import the zoo module as menagerie and call its hours() function.

In [6]:
import zoo as menagerie
menagerie.hours()

Open 9-5 daily.


16.4: This isn't listed as a requirement in our module, but 16.8 requires this to be done

16.4 Use the sqlite3 module to create a SQLite database called books.db and a table called books with these fields: title (text), author (text), and year (integer).

In [6]:
import sqlite3
con = sqlite3.connect("books.db")
#SQLite needs a cursor object to interact with the database.
cur = con.cursor()
#Create our table
cur.execute("""CREATE TABLE books(
            title text, 
            author text, 
            year integer)
""")


OperationalError: table books already exists

16.8 Use the sqlalchemy module to connect to the sqlite3 database books.db that you just made in exercise 16.4. As in 16.6, select and print the title column from the book table in alphabetical order.


In [3]:
!pip install sqlalchemy

  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/d0/ff/5746886a796799d45285573c8a6564b4b84c730142ab974d7a3f7bacee6c/SQLAlchemy-2.0.31-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.6.0 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/53/80/3d94d5999b4179d91bcc93745d1b0815b073d61be79dd546b840d17adb18/greenlet-3.0.3-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 991.0 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.1 MB 3.8 MB/s eta 0:00:01
   ------------ --------------------------- 0.6/2.1 MB 5.0 MB/s eta 0:00:01
   ------------------ -------------------


[notice] A new release of pip is available: 23.2.1 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
import sqlalchemy as sa
import csv

#Create our engine and connect to the books database.
engine = sa.create_engine('sqlite:///books.db')

#Create metadata since our table already exists.
meta = sa.MetaData()

#Call the table method and plug in our connection to autoload_with since our database already exists.
books = sa.Table('books', meta, autoload_with=engine)

#Data to write to the csv file from our textbook.
write_data = """
title,author,year
The Weirdstone of Brisingamen,Alan Garner,1960
Perdido Street Station,China Miéville,2000
Thud!,Terry Pratchett,2005
The Spellman Files,Lisa Lutz,2007
Small Gods,Terry Pratchett,1992
"""

#Return an iterable list delimited by the newline separator
lines = write_data.strip().split('\n')

#Write to our csv file
with open('books2.csv', 'w', newline='') as f:
    writer=csv.writer(f)
    #Iterate over our lines list
    for line in lines:
        #Our delimiter per line is the comma. 
        writer.writerow(line.split(','))


#Read our CSV file and store it in a list
with open('books2.csv', 'rt', newline='') as f:
    reader = csv.DictReader(f) 
    results = [row for row in reader]

#Insert our csv data into the database
with engine.connect() as connection:
    for row in results:
        connection.execute(books.insert(), row)
    
    #Generate a sql statement to query, and return alphabetical titles.
    stmt = sa.select(sa.func.concat("Title: ", books.c.title).label("title")).select_from(books).order_by(books.c.title)
    
    results = connection.execute(stmt).fetchall()
    for result in results:
        print(result.title)

Title: Perdido Street Station
Title: Small Gods
Title: The Spellman Files
Title: The Weirdstone of Brisingamen
Title: Thud!
